## Set up matching DF's

In [356]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)
import os
import re

In [357]:
directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/Toto"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_toto = max(files, key=os.path.getmtime)

directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/unibet"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_unibet = max(files, key=os.path.getmtime)

In [358]:
latest_file_toto

'/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/Toto/totoAllSport2025-02-09 10:14:38.346593.csv'

In [359]:
toto_raw = pd.read_csv(f"{latest_file_toto}", index_col=0).drop_duplicates()
kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw_football = toto_raw[toto_raw['sport'] == 'Voetbal']
kambi_raw_football = kambi_raw[kambi_raw['sport'] == 'FOOTBALL']
# Adjust odds and line
kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
kambi_raw_football['odds'] = kambi_raw_football['odds'] / 1000

# Adjust toto outcome
toto_raw_football['Outcome SubType'] = toto_raw_football['Outcome SubType'].replace({'H': '1', 'A': '2'})

# List of women's competitions
women_competitions_toto = [
    'Portugal Campeonato Nacional, Vrouwen', 'Mexico League MX Vrouwen', 'Australië W-League', 'Italië Coppa Italia Vrouwen',
    'Scotland Women\'s Premier League', 'Nederland Eredivisie Vrouwen', 'England FA Cup Women', 'Engeland FA Super League Vrouwen',
    'Spain Primera División Vrouwen'
]

# Create 'sex' column based on the competition
toto_raw_football['sex'] = toto_raw_football['competition'].apply(
    lambda x: 'W' if x in women_competitions_toto else 'M'
)

# List of women's football competitions
women_competitions_kambi = [
    'A-League (D)', 'Premier League Dames', 'Campeonato Nacional Feminino', 'Liga MX Femenil (D)', 
    'Frauen-Bundesliga', 'Super League (D)', 'Primera División (D)', 'Coppa Italia (D)', 'Liga MX Femenil'

]

# Create 'sex' column
kambi_raw_football['sex'] = kambi_raw_football['group_name'].apply(
    lambda x: 'W' if x in women_competitions_kambi else 'M'
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/3251815356.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/3251815356.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/3251815356.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [360]:
toto_raw

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time
event_id,,,,,,,,,,,,,,
6816915,Oxford United vs Portsmouth,1e Helft: Resultaat,Gelijkspel,2.24,31,25,H1,D,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z
6816915,Oxford United vs Portsmouth,1e Helft: Resultaat,Oxford United,3.05,41,20,H1,H,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z
6816915,Oxford United vs Portsmouth,1e Helft: Resultaat,Portsmouth,3.30,23,10,H1,A,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z
6816915,Oxford United vs Portsmouth,Beide Teams Scoren,Ja,1.61,61,100,--,NaN,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z
6816915,Oxford United vs Portsmouth,Beide Teams Scoren,Nee,2.18,59,50,--,NaN,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6816585,August Holmgren vs Shintaro Mochizuki,1e Set - Wedstrijd Resultaat,Shintaro Mochizuki / August Holmgren,8.75,31,4,--,NaN,Tennis,"ATP Challenger New Delhi, Enkelspel",August Holmgren vs Shintaro Mochizuki,August Holmgren,Shintaro Mochizuki,2025-02-11T04:30:00Z
6816585,August Holmgren vs Shintaro Mochizuki,1e Set - Wedstrijd Resultaat,August Holmgren / August Holmgren,2.50,6,4,--,NaN,Tennis,"ATP Challenger New Delhi, Enkelspel",August Holmgren vs Shintaro Mochizuki,August Holmgren,Shintaro Mochizuki,2025-02-11T04:30:00Z
6816585,August Holmgren vs Shintaro Mochizuki,1e Set - Wedstrijd Resultaat,August Holmgren / Shintaro Mochizuki,7.75,27,4,--,NaN,Tennis,"ATP Challenger New Delhi, Enkelspel",August Holmgren vs Shintaro Mochizuki,August Holmgren,Shintaro Mochizuki,2025-02-11T04:30:00Z


In [361]:
toto_raw[toto_raw['Market Name'].str.contains('Sergi Gomez aantal schoten op doel')]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time
event_id,,,,,,,,,,,,,,
6019542,Real Sociedad vs Espanyol,Sergi Gomez aantal schoten op doel (Inc Extra ...,3+,60.0,59,1,--,NaN,Voetbal,Spanje LaLiga,Real Sociedad vs Espanyol,Real Sociedad,Espanyol,2025-02-09T17:30:00Z
6019542,Real Sociedad vs Espanyol,Sergi Gomez aantal schoten op doel (Inc Extra ...,2+,23.0,22,1,--,NaN,Voetbal,Spanje LaLiga,Real Sociedad vs Espanyol,Real Sociedad,Espanyol,2025-02-09T17:30:00Z
6019542,Real Sociedad vs Espanyol,Sergi Gomez aantal schoten op doel (Inc Extra ...,5+,250.0,249,1,--,NaN,Voetbal,Spanje LaLiga,Real Sociedad vs Espanyol,Real Sociedad,Espanyol,2025-02-09T17:30:00Z
6019542,Real Sociedad vs Espanyol,Sergi Gomez aantal schoten op doel (Inc Extra ...,1+,4.5,7,2,--,NaN,Voetbal,Spanje LaLiga,Real Sociedad vs Espanyol,Real Sociedad,Espanyol,2025-02-09T17:30:00Z
6019542,Real Sociedad vs Espanyol,Sergi Gomez aantal schoten op doel (Inc Extra ...,4+,150.0,149,1,--,NaN,Voetbal,Spanje LaLiga,Real Sociedad vs Espanyol,Real Sociedad,Espanyol,2025-02-09T17:30:00Z


Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [362]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_filtered_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_football based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name', 'Odds (Decimal)']
)
# Remove duplicates in kambi_filtered_football based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label', 'line', 'odds']
)

# Replace 'Sport Club' with 'SC' in the 'event_name' column
kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/1733036263.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)


In [363]:
kambi_filtered_football[kambi_filtered_football['criterion_label'].str.contains('Speler')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex
13256,2489082522,1004544421,Schoten van Speler (Afgehandeld Volgens Opta-G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1021150789,3624994843,Meer dan,Over,4.40,1.5,"Fernandes da Silva Junior, Bernardo",OT_OVER,2025-02-09T09:14:17Z,17/5,340.0,OPEN,ENABLED,NaN,NaN,Holstein Kiel vs VfL Bochum,FOOTBALL,Bundesliga,2025-02-09T14:30:00Z,M
13257,2489082524,1004544421,Schoten van Speler (Afgehandeld Volgens Opta-G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1021150789,3624994847,Meer dan,Over,1.70,2.5,"Harres, Phil",OT_OVER,2025-02-09T09:14:17Z,7/10,-143.0,OPEN,ENABLED,NaN,NaN,Holstein Kiel vs VfL Bochum,FOOTBALL,Bundesliga,2025-02-09T14:30:00Z,M
13262,2489082528,1004544421,Schoten van Speler (Afgehandeld Volgens Opta-G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1021150789,3624994855,Meer dan,Over,1.80,0.5,"Fernandes da Silva Junior, Bernardo",OT_OVER,2025-02-09T09:14:17Z,4/5,-125.0,OPEN,ENABLED,NaN,NaN,Holstein Kiel vs VfL Bochum,FOOTBALL,Bundesliga,2025-02-09T14:30:00Z,M
13264,2489082530,1004544421,Schoten van Speler (Afgehandeld Volgens Opta-G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1021150789,3624994859,Meer dan,Over,1.33,0.5,"Porath, Finn",OT_OVER,2025-02-09T09:14:17Z,33/100,-305.0,OPEN,ENABLED,NaN,NaN,Holstein Kiel vs VfL Bochum,FOOTBALL,Bundesliga,2025-02-09T14:30:00Z,M
13269,2489082537,1004544421,Schoten van Speler (Afgehandeld Volgens Opta-G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1021150789,3624994873,Meer dan,Over,1.26,0.5,"Bero, Matús",OT_OVER,2025-02-09T09:14:16Z,1/4,-385.0,OPEN,ENABLED,NaN,NaN,Holstein Kiel vs VfL Bochum,FOOTBALL,Bundesliga,2025-02-09T14:30:00Z,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98095,2491870323,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022456405,3635387569,Minder dan,Under,2.30,0.5,Lucas Moura,OT_UNDER,2025-02-06T15:58:15Z,13/10,130.0,OPEN,ENABLED,NaN,NaN,São Paulo-SP vs Inter de Limeira-SP,FOOTBALL,Paulista A1,2025-02-11T00:30:00Z,M
98103,2491870332,1004544421,Schoten van Speler (Afgehandeld Volgens Opta-G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022456405,3635387554,Meer dan,Over,1.34,1.5,Luciano da Rocha,OT_OVER,2025-02-06T15:58:15Z,1/3,-295.0,OPEN,ENABLED,NaN,NaN,São Paulo-SP vs Inter de Limeira-SP,FOOTBALL,Paulista A1,2025-02-11T00:30:00Z,M
98144,2491870370,1004544421,Schoten van Speler (Afgehandeld Volgens Opta-G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022456405,3635387930,Meer dan,Over,2.23,3.5,"Calleri, Jonathan",OT_OVER,2025-02-06T15:58:15Z,6/5,123.0,OPEN,ENABLED,NaN,NaN,São Paulo-SP vs Inter de Limeira-SP,FOOTBALL,Paulista A1,2025-02-11T00:30:00Z,M
98155,2491870381,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022456405,3635387964,Meer dan,Over,4.10,0.5,"Arboleda, Robert",OT_OVER,2025-02-06T15:58:16Z,3/1,310.0,OPEN,ENABLED,NaN,NaN,São Paulo-SP vs Inter de Limeira-SP,FOOTBALL,Paulista A1,2025-02-11T00:30:00Z,M


## Winnaar: Draw no Bet

In [364]:
from rapidfuzz import process, fuzz
import pandas as pd

# Filter kambi_filtered_football based on criterion_label
filtered_kambi_winnaar = kambi_filtered_football[
    kambi_filtered_football['criterion_label'].str.contains('Draw No Bet')
]

# Filter kambi_filtered_football based on criterion_label
filtered_toto_winnaar = toto_filtered_football[
    toto_filtered_football['Market Name'].str.contains('Draw No Bet')
]

# Preprocess strings: Remove extra whitespace and lowercase
def preprocess(text):
    return ' '.join(text.lower().split())

In [365]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match based on Team1 and Team2 fuzzy matching
# def find_best_match(event_name):
#     # Extract Team1 and Team2 from the event_name
#     parts = event_name.split(' vs ')
#     if len(parts) == 2:
#         team1 = parts[0].strip()
#         team2 = parts[1].strip()

#         # Perform fuzzy matching for Team1 and Team2
#         def match_teams(team, kambi_team):
#             return fuzz.token_set_ratio(team, kambi_team) >= 80

#         # Check each event_name in kambi_events
#         for kambi_event in kambi_events:
#             kambi_parts = kambi_event.split(' vs ')
#             if len(kambi_parts) == 2:
#                 kambi_team1 = kambi_parts[0].strip()
#                 kambi_team2 = kambi_parts[1].strip()

#                 # Check if both teams match
#                 if match_teams(team1, kambi_team1) and match_teams(team2, kambi_team2):
#                     return kambi_event  # Return the matched event_name

#     return None  # Return None if no match found

from fuzzywuzzy import fuzz

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(event_name):
    # Extract Team1 and Team2 from the event_name
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip().replace('Athletic', '').replace('Atletico', '')
        team2 = parts[1].strip().replace('Athletic', '').replace('Atletico', '')

        # Perform fuzzy matching for Team1 and Team2 with all kambi events
        def match_teams(team, kambi_team):
            return fuzz.token_set_ratio(team, kambi_team)

        # Perform fuzzy matching for both Team1 and Team2
        kambi_matches = filtered_kambi_winnaar[
            (filtered_kambi_winnaar['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
            (filtered_kambi_winnaar['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
        ]
        
        # If there is a match, return the matched event_name and fuzzy match score
        if not kambi_matches.empty:
            # Get the best match score
            best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
            best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()

            # Calculate the average score (or you can choose to return another metric)
            average_score = (best_score_team1 + best_score_team2) / 2

            # Get the matched event name
            matched_event_name = kambi_matches['event_name'].iloc[0]

            return matched_event_name, average_score  # Return the matched event and score

    return None, None  # Return None if no match found

# Apply matching function to toto_filtered_football
filtered_toto_winnaar[['matched_event', 'fuzzy_score']] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: pd.Series(find_best_match(x))
)

# # Apply matching function to filtered_toto_winnaar
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

# Check the results
filtered_toto_winnaar[['Event Name', 'matched_event']].head()

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/1566042253.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/1566042253.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/1566042253.py:20: SettingW

,Event Name,matched_event
event_id,,
6816915,Oxford United vs Portsmouth,None
6816915,Oxford United vs Portsmouth,None
6816915,Oxford United vs Portsmouth,None
6816915,Oxford United vs Portsmouth,None
6816920,Preston North End vs Burnley,None


In [366]:
filtered_kambi_winnaar

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,Team1,Team2
6410,2492591396,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022762420,3638433227,1,1,1.14,NaN,San Lorenzo De Almagro (D),1,2025-02-09T04:53:15Z,2/15,-715.0,OPEN,DISABLED,NaN,NaN,San Lorenzo (W) vs Rosario Central (W),FOOTBALL,Femenino (D),2025-02-09T12:00:00Z,M,San Lorenzo (W),Rosario Central (W)
6411,2492591396,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022762420,3638433230,2,2,4.90,NaN,Rosario Central (D),2,2025-02-09T04:53:15Z,19/5,390.0,OPEN,DISABLED,NaN,NaN,San Lorenzo (W) vs Rosario Central (W),FOOTBALL,Femenino (D),2025-02-09T12:00:00Z,M,San Lorenzo (W),Rosario Central (W)
6553,2490811820,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022623464,3631358069,1,1,1.77,NaN,Deportivo Riestra,1,2025-02-08T08:05:55Z,3/4,-130.0,OPEN,DISABLED,NaN,NaN,Deportivo Riestra vs Defensa y Justicia,FOOTBALL,Liga Profesional Argentina,2025-02-09T20:00:00Z,M,Deportivo Riestra,Defensa y Justicia
6554,2490811820,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022623464,3631358081,2,2,1.91,NaN,Defensa y Justicia,2,2025-02-08T08:05:55Z,10/11,-110.0,OPEN,DISABLED,NaN,NaN,Deportivo Riestra vs Defensa y Justicia,FOOTBALL,Liga Profesional Argentina,2025-02-09T20:00:00Z,M,Deportivo Riestra,Defensa y Justicia
6594,2490811842,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022623464,3631358150,1,1,1.79,NaN,Deportivo Riestra,1,2025-02-08T08:05:54Z,10/13,-127.0,OPEN,DISABLED,NaN,NaN,Deportivo Riestra vs Defensa y Justicia,FOOTBALL,Liga Profesional Argentina,2025-02-09T20:00:00Z,M,Deportivo Riestra,Defensa y Justicia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103263,2491788933,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022715874,3635123941,2,2,3.45,NaN,Portadown,2,2025-02-06T08:41:32Z,49/20,245.0,OPEN,DISABLED,NaN,NaN,Cliftonville vs Portadown,FOOTBALL,NIFL Premiership,2025-02-09T15:00:00Z,M,Cliftonville,Portadown
103328,2491788954,1001421321,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2,Wedstrijd,Match,1022715874,3635124023,1,1,1.34,NaN,Cliftonville,1,2025-02-06T08:41:32Z,1/3,-295.0,OPEN,ENABLED,NaN,NaN,Cliftonville vs Portadown,FOOTBALL,NIFL Premiership,2025-02-09T15:00:00Z,M,Cliftonville,Portadown
103329,2491788954,1001421321,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2,Wedstrijd,Match,1022715874,3635124028,2,2,3.00,NaN,Portadown,2,2025-02-06T08:41:32Z,2/1,200.0,OPEN,ENABLED,NaN,NaN,Cliftonville vs Portadown,FOOTBALL,NIFL Premiership,2025-02-09T15:00:00Z,M,Cliftonville,Portadown
103389,2491788978,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022715874,3635124130,1,1,1.37,NaN,Cliftonville,1,2025-02-06T08:41:33Z,4/11,-275.0,OPEN,ENABLED,NaN,NaN,Cliftonville vs Portadown,FOOTBALL,NIFL Premiership,2025-02-09T15:00:00Z,M,Cliftonville,Portadown


In [367]:
# Get unique records from 'Event Name' and 'matched_event'
matched_events = filtered_toto_winnaar[['Event Name', 'matched_event', 'fuzzy_score']].drop_duplicates()

# Display the result
matched_events[~matched_events['matched_event'].isna()].sort_values(by='fuzzy_score', ascending=False).to_csv('test_match.csv')

In [368]:
# Define a transformation function for standardization
def standardize_draw_no_bet(value):
    if '1e Helft' in value:
        return 'Draw No Bet - 1e Helft'
    elif '2e Helft' in value:
        return 'Draw No Bet - 2e Helft'
    elif 'Draw No Bet' in value:
        return 'Draw No Bet'
    else:
        return value  # Keep original if no match

# Apply the transformation to the relevant columns
filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)

# Merge the DataFrames using the standardized column
merged_winnaar = pd.merge(
    filtered_toto_winnaar,
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label'], 
    right_on=['event_name', 'standardized_label'], 
    how='inner'  # Perform an inner join
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/3072039294.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/3072039294.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)


In [369]:
# # Filter rows in toto_filtered_football where Market Name == 'Winnaar'
# toto_filtered_football = toto_filtered_football[toto_filtered_football['Market Name'].str.contains('Draw No Bet')]

# Perform a left join on the matched_event_name (from toto) and event_name (from kambi)
merged_df_winnaar = filtered_toto_winnaar.merge(
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label', 'sex', 'start_time'], 
    right_on=['event_name', 'standardized_label', 'sex', 'start_time'], 
    how='left'
)

merged_df_winnaar = merged_df_winnaar[(merged_df_winnaar['matched_event'].notnull())&(merged_df_winnaar['Outcome SubType'] != merged_df_winnaar['outcome_label'])]

In [370]:
merged_df_winnaar

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,start_time,sex,Team1_x,Team2_x,matched_event,fuzzy_score,standardized_label,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y
6,AZ Alkmaar vs Fortuna Sittard,Draw No Bet,AZ Alkmaar,1.14,7,50,DN,1,Voetbal,Nederland Eredivisie Vrouwen,AZ Alkmaar vs Fortuna Sittard,AZ Alkmaar,Fortuna Sittard,2025-02-09T15:45:00Z,W,AZ Alkmaar,Fortuna Sittard,AZ Alkmaar (W) vs Fortuna Sittard (W),100.0,Draw No Bet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AZ Alkmaar vs Fortuna Sittard,Draw No Bet,Fortuna Sittard,4.00,3,1,DN,2,Voetbal,Nederland Eredivisie Vrouwen,AZ Alkmaar vs Fortuna Sittard,AZ Alkmaar,Fortuna Sittard,2025-02-09T15:45:00Z,W,AZ Alkmaar,Fortuna Sittard,AZ Alkmaar (W) vs Fortuna Sittard (W),100.0,Draw No Bet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AZ Alkmaar vs Fortuna Sittard,2e Helft: Draw No Bet,Fortuna Sittard,3.15,43,20,DN,2,Voetbal,Nederland Eredivisie Vrouwen,AZ Alkmaar vs Fortuna Sittard,AZ Alkmaar,Fortuna Sittard,2025-02-09T15:45:00Z,W,AZ Alkmaar,Fortuna Sittard,AZ Alkmaar (W) vs Fortuna Sittard (W),100.0,Draw No Bet - 2e Helft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AZ Alkmaar vs Fortuna Sittard,2e Helft: Draw No Bet,AZ Alkmaar,1.23,23,100,DN,1,Voetbal,Nederland Eredivisie Vrouwen,AZ Alkmaar vs Fortuna Sittard,AZ Alkmaar,Fortuna Sittard,2025-02-09T15:45:00Z,W,AZ Alkmaar,Fortuna Sittard,AZ Alkmaar (W) vs Fortuna Sittard (W),100.0,Draw No Bet - 2e Helft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,AZ Alkmaar vs Fortuna Sittard,1e Helft: Draw No Bet,AZ Alkmaar,1.22,11,50,DN,1,Voetbal,Nederland Eredivisie Vrouwen,AZ Alkmaar vs Fortuna Sittard,AZ Alkmaar,Fortuna Sittard,2025-02-09T15:45:00Z,W,AZ Alkmaar,Fortuna Sittard,AZ Alkmaar (W) vs Fortuna Sittard (W),100.0,Draw No Bet - 1e Helft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5694,TS Galaxy FC vs Mamelodi Sundowns,2e Helft: Draw No Bet,TS Galaxy FC,3.15,43,20,DN,1,Voetbal,Zuid-Afrika Premier Soccer League,TS Galaxy FC vs Mamelodi Sundowns,TS Galaxy FC,Mamelodi Sundowns,2025-02-11T17:30:00Z,M,TS Galaxy FC,Mamelodi Sundowns,TS Galaxy FC vs Mamelodi Sundowns,100.0,Draw No Bet - 2e Helft,2.490960e+09,1.001421e+09,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2.0,Wedstrijd,Match,1.022499e+09,3.632104e+09,2,2,1.25,NaN,Mamelodi Sundowns,2,2025-02-09T00:49:47Z,1/4,-400.0,OPEN,DISABLED,NaN,NaN,TS Galaxy FC vs Mamelodi Sundowns,FOOTBALL,SA Premier Soccer League,TS Galaxy FC,Mamelodi Sundowns
5755,FC Uta Arad vs CFR Cluj,2e Helft: Draw No Bet,CFR Cluj,1.44,11,25,DN,2,Voetbal,Roemenië Liga 1,FC Uta Arad vs CFR Cluj,FC Uta Arad,CFR Cluj,2025-02-10T18:00:00Z,M,FC Uta Arad,CFR Cluj,AFC UTA Arad vs CFR Cluj,98.0,Draw No Bet - 2e Helft,2.491084e+09,1.001421e+09,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2.0,Wedstrijd,Match,1.021130e+09,3.632341e+09,1,1,2.55,NaN,AFC UTA Arad,1,2025-02-06T23:43:18Z,31/20,155.0,OPEN,DISABLED,NaN,NaN,AFC UTA Arad vs CFR Cluj,FOOTBALL,Liga I,AFC UTA Arad,CFR Cluj
5758,FC Uta Arad vs CFR Cluj,2e H

## Over/Under

In [371]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_raw_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [372]:
kambi_filtered_football_overunder = kambi_filtered_football[(kambi_filtered_football['outcome_english_label'].str.contains('Over')) |
    (kambi_filtered_football['outcome_english_label'].str.contains('Under'))
]
toto_filtered_football_overunder = toto_filtered_football[
    (toto_filtered_football['Outcome Name'].str.contains('Over')) |
    (toto_filtered_football['Outcome Name'].str.contains('Under')) |
    (toto_filtered_football['Outcome Name'].str.match(r'^\d+\+$')) |  # matches strings that ONLY contain digit(s) followed by '+'
    (toto_filtered_football['Outcome Name'].str.contains(r'\d{1,2} of meer')) &
    ~(toto_filtered_football['Outcome Name'].str.contains(' en ')) &  # excludes strings containing ' en '
    ~(toto_filtered_football['Outcome Name'].str.contains('&'))  # excludes strings containing '&'
]

In [373]:
toto_filtered_football_overunder[(toto_filtered_football_overunder['Event Name'].str.contains('Rakow Czestochowa vs GKS Katowice')) & (toto_filtered_football_overunder['Market Name'].str.contains('Aantal Goals   Over/Under 1.5'))]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex
event_id,,,,,,,,,,,,,,,


In [374]:
kambi_raw_football[(kambi_raw_football['event_id']==1020968352)& (kambi_raw_football['criterion_label']=='Totaal Aantal Doelpunten')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex


In [375]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_english_label'] = kambi_filtered_football_overunder['criterion_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Event Name'] = toto_filtered_football_overunder['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Market Name'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = kambi_filtered_football_overunder['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score = result
#     return match


# # Apply matching function to toto_filtered_football
# toto_filtered_football_overunder['matched_event'] = toto_filtered_football_overunder['Event Name'].apply(find_best_match)
# Perform a left join to bring in 'matched_event' and 'fuzzy_score' from matched_events
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    matched_events,
    on='Event Name',  # Join on the 'Event Name' column
    how='left'  # Ensure all rows in toto_filtered_football_overunder are retained
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/2219985270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/2219985270.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_r

In [376]:
matched_events[matched_events['Event Name'].str.contains('Rakow Czestochowa vs GKS Katowice')]

,Event Name,matched_event,fuzzy_score
event_id,,,


In [377]:
# Get unique records from 'Event Name' and 'matched_event'
unique_records = toto_filtered_football_overunder[['Event Name', 'matched_event']].drop_duplicates()

# Display the result
unique_records[~unique_records['matched_event'].isna()]

,Event Name,matched_event
113,AZ Alkmaar vs Fortuna Sittard,AZ Alkmaar (W) vs Fortuna Sittard (W)
187,NAC vs Feyenoord,NAC Breda vs Feyenoord
255,Leicester City vs Brentford,Leicester City vs Brentford
285,Ipswich Town vs Tottenham Hotspur,Ipswich Town vs Tottenham
409,Ajax vs Heracles Almelo,Ajax vs Heracles Almelo
...,...,...
61478,Sportivo Ameliano vs CS 2 De Mayo,Sportivo Ameliano vs 2 de Mayo
61819,FCSB vs ACS Sepsi Osk Sfantu Gheorghe,FCSB vs ACS Sepsi OSK Sfantul Gheorghe
62276,FC Uta Arad vs CFR Cluj,AFC UTA Arad vs CFR Cluj
62616,Sekhukhune United vs Supersport United,Sekhukhune United vs Supersport United


In [378]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')  

In [379]:
# Create 'OverUnderType'
kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
        lambda x: 'Goals' if ('Doelpunten' in x and 'Resultaat' not in x and 'Doelpuntenmaker' not in x) 
        else 'Team schoten op doel' if ('Totaal Aantal Schoten op Doel door' in x and ' & ' not in x) 
        else 'Wedstrijd schoten op doel' if ('Totaal Aantal Schoten op Doel' in x and ' & ' not in x)
        else 'Team schoten' if ('Totaal Aantal Schoten door' in x and ' & ' not in x) 
        else 'Wedstrijd schoten' if ('Totaal Aantal Schoten' in x and ' & ' not in x)
        else 'Speler schoten op doel' if ('Schoten van Speler op Doel' in x and ' & ' not in x)
        else 'Speler schoten' if ('Schoten van Speler' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
         else 'Dubbele Kans' if ('Dubbele Kans' in x or ' en ' in x or ' & ' in x)
        else 'other'
    )

def determine_over_under_time(label):
    label_lower = label.lower()

    if '1e helft' in label_lower:
        return '1e Helft'
    elif '2e helft' in label_lower:
        return '2e Helft'

    # Check if the label contains a time range in the format 'XX:XX YY:YY'
    time_match = re.search(r'\b\d{1,2}:\d{2} \d{1,2}:\d{2}\b', label)
    if time_match:
        return time_match.group()  # Return the matched time range
    
    return 'Full Time'

# Apply the function to the column
kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

# Create 'Team1' and 'Team2'
kambi_filtered_football_overunder[['Team1', 'Team2']] = kambi_filtered_football_overunder['event_name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
    lambda row: row['participant'] if row['OverUnderType'] in ['Speler schoten op doel', 'Speler schoten'] else (
        '1' if row['Team1'] in row['criterion_english_label'] else 
        '2' if row['Team2'] in row['criterion_english_label'] else 
        'Total team 1 and team 2'
    ),
    axis=1
)

# # Create 'OverUnderType2'
# kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
#     lambda row: '1' if row['Team1'] in row['criterion_english_label'] else (
#         '2' if row['Team2'] in row['criterion_english_label'] else 'Total team 1 and team 2'
#     ),
#     axis=1
# )

# # Create 'line'
# kambi_filtered_football_overunder['line'] = kambi_filtered_football_overunder['line'] / 1000

# # Creat odds
# kambi_filtered_football_overunder['odds'] = kambi_filtered_football_overunder['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/3258436082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_69424/3258436082.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)
/va

In [380]:
kambi_filtered_football_overunder[(kambi_filtered_football_overunder['OverUnderTime'] != '1e Helft') & (kambi_filtered_football_overunder['OverUnderTime'] != '2e Helft') & (kambi_filtered_football_overunder['OverUnderTime'] != 'Full Time')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2


In [381]:
# Create 'OverUnderType'
toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: 'Goals' if ('Goals' in x and 'Resultaat' not in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    # else 'Speler schoten op doel' if ('Speler schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    else 'Wedstrijd schoten op doel' if ('Wedstrijd schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    else 'Team schoten op doel' if ('Team schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    # else 'Schoten op doel van buiten 16 mtr' if ('schoten op doel van buiten 16 mtr' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) # via specials voor later
    else 'Speler schoten op doel' if ('aantal schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) or ('Speler schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    else 'Speler schoten' if ('aantal schoten' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    else 'Dubbele Kans' if ('Dubbele Kans' in x or ' en ' in x or ' & ' in x)
    else 'other'
)

# Create 'OverUnderTime'
toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: '1e Helft' if '1e helft' in x.lower() else '2e Helft' if '2e helft' in x.lower() 
    else  '00:00 09:59' if 'eerste 10 minuten' in x.lower() else 'Full Time'
)

# Create 'Team1' and 'Team2'
toto_filtered_football_overunder[['Team1', 'Team2']] = toto_filtered_football_overunder['Event Name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
    lambda row: (
        '1' if row['OverUnderType'] == 'Goals' and row['Team1'] in row['Market Name'] else
        '2' if row['OverUnderType'] == 'Goals' and row['Team2'] in row['Market Name'] else
        'Total team 1 and team 2' if row['OverUnderType'] == 'Goals' else
        'Total team 1 and team 2' if row['OverUnderType'] == 'Wedstrijd schoten op doel' else
        '1' if row['OverUnderType'] == 'Team schoten op doel' and row['Team1'] in row['Outcome Name'] else
        '2' if row['OverUnderType'] == 'Team schoten op doel' and row['Team2'] in row['Outcome Name'] else
        row['Market Name'].split('aantal schoten')[0].strip() if row['OverUnderType'] in ['Speler schoten op doel', 'Speler schoten'] 
        else 'other'  # Default to OverUnderType if none of the conditions apply
    ),
    axis=1
)


def extract_line_toto(row):
    market_name = row['Market Name']
    outcome_name = row['Outcome Name']
    over_under_type = row['OverUnderType']

    # Extract number from 'Market Name' when it contains 'Over/Under '
    if 'Over/Under ' in market_name:
        return float(market_name.split('Over/Under ')[-1])

    # Extract number from 'Outcome Name'
    match = re.search(r'\d+(\.\d+)?', outcome_name)
    if match:
        value = float(match.group())  # Convert extracted number to float

        # Adjust value based on conditions
        if over_under_type in ['Wedstrijd schoten op doel', 'Team schoten op doel'] and 'of meer' in outcome_name:
            return value - 0.5
        elif over_under_type in ['Speler schoten op doel', 'Speler schoten'] and '+' in outcome_name:
            return value - 0.5
        else:
            return value

    return None  # Default if no number found

# Apply function to create 'line' column
toto_filtered_football_overunder['line'] = toto_filtered_football_overunder.apply(extract_line_toto, axis=1)

# Adjust Outcome Name where {digit} + to over
toto_filtered_football_overunder['Outcome Name'] = toto_filtered_football_overunder['Outcome Name'].apply(
    lambda outcome_name: 'Over' if re.search(r'\d+(\.\d+)?\+', outcome_name) else outcome_name
)

In [382]:
# # Function to find the best fuzzy match for OverUnderType2
# def find_best_fuzzy_match(overunder_type2, kambi_df):
#     # Perform fuzzy matching with all Kambi OverUnderType2 values
#     match_scores = kambi_df['OverUnderType2'].apply(lambda x: fuzz.token_set_ratio(overunder_type2, x))
    
#     # Get the best match score and the corresponding OverUnderType2 value
#     best_match_idx = match_scores.idxmax()
#     best_score = match_scores.max()
    
#     if best_score >= 80:  # Threshold for fuzzy matching
#         return kambi_df.loc[best_match_idx, 'OverUnderType2'], best_score
#     return None, None  # No match found

# # Filter records where OverUnderType contains 'Team' or 'Speler'
# filtered_toto = toto_filtered_football_overunder[
#     toto_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]
# filtered_kambi = kambi_filtered_football_overunder[
#     kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]

# # Apply fuzzy matching function
# filtered_toto[['matched_OverUnderType2', 'fuzzy_score']] = filtered_toto['OverUnderType2'].apply(
#     lambda x: pd.Series(find_best_fuzzy_match(x, filtered_kambi))
# )

# # Merge back to the original dataset (if needed)
# toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
#     filtered_toto[['OverUnderType2', 'matched_OverUnderType2', 'fuzzy_score']],
#     on='OverUnderType2',
#     how='left'
# )

# # Fill OverUnderType2 with matched_OverUnderType2 if it is null then OverUnderType2
# toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
#     lambda row: row['OverUnderType2'] if pd.isna(row['matched_OverUnderType2']) else row['matched_OverUnderType2'], axis=1
# )

In [383]:
# Function to find the best fuzzy match for OverUnderType2
def find_best_fuzzy_match(overunder_type2, kambi_df):
    # Perform fuzzy matching with all Kambi OverUnderType2 values
    match_scores = kambi_df['OverUnderType2'].apply(lambda x: fuzz.token_set_ratio(str(overunder_type2), str(x)))
    
    # Get the best match score and the corresponding OverUnderType2 value
    if len(match_scores) > 0:
        best_match_idx = match_scores.idxmax()
        best_score = match_scores.max()
        
        if best_score >= 80:  # Threshold for fuzzy matching
            return kambi_df.loc[best_match_idx, 'OverUnderType2'], best_score
    return None, None  # No match found

# Filter records
filtered_toto = toto_filtered_football_overunder[
    toto_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
].copy()

filtered_kambi = kambi_filtered_football_overunder[
    kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
].copy()

# Apply fuzzy matching function
fuzzy_matches = filtered_toto['OverUnderType2'].apply(
    lambda x: find_best_fuzzy_match(x, filtered_kambi)
)

filtered_toto['matched_OverUnderType2'] = fuzzy_matches.apply(lambda x: x[0])
filtered_toto['fuzzy_score'] = fuzzy_matches.apply(lambda x: x[1])

# Merge back to the original dataset
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    filtered_toto[['OverUnderType2', 'matched_OverUnderType2', 'fuzzy_score']],
    on='OverUnderType2',
    how='left'
)

# Fill OverUnderType2 with matched_OverUnderType2 if it is null
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder['matched_OverUnderType2'].fillna(toto_filtered_football_overunder['OverUnderType2'])

In [384]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')
# merged_football_overunder.to_csv('Data/Testing/merged_football_overunder.csv')


In [385]:
# Perform the merge
merged_football_overunder = pd.merge(
    toto_filtered_football_overunder,
    kambi_filtered_football_overunder,
    left_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'matched_event', 'sex', 'start_time'],
    right_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'event_name', 'sex', 'start_time'],
    how='inner'  # Use 'inner' join to get only matching rows; adjust as needed
)

merged_football_overunder['Outcome Name cleaned'] = merged_football_overunder['Outcome Name'].apply(
    lambda x: 'Over' if re.search(r'\b\d{1,2} of meer\b', x) else x
)

# Keep records with opposite outcomes
merged_football_overunder = merged_football_overunder[merged_football_overunder['outcome_english_label'] != merged_football_overunder['Outcome Name cleaned']] # Dubbel check voor Schoten OverUnderType & voer fuzzymatching uit op speler.

# Display the merged DataFrame
merged_football_overunder.head()

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score_x,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,matched_OverUnderType2,fuzzy_score_y,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y,Outcome Name cleaned
0,NAC vs Feyenoord,Feyenoord 1e Helft: Aantal Goals Over/Under 0.5,Under,2.41,141,100,HL,L,Voetbal,Nederland Eredivisie,NAC vs Feyenoord,NAC,Feyenoord,2025-02-15T17:45:00Z,M,NAC Breda vs Feyenoord,100.0,Goals,1e Helft,NAC,Feyenoord,2,0.5,NaN,NaN,2492348909,1003194956,Totaal Aantal Doelpunten door Feyenoord 1e H...,Total Goals by Feyenoord 1st Half,NaN,NaN,6,Over/Onder,Over/Under,1021049420,3637367915,Meer dan,Over,1.46,NaN,OT_OVER,2025-02-09T07:44:04Z,4/9,-220.0,OPEN,ENABLED,NaN,NaN,NAC Breda vs Feyenoord,FOOTBALL,Eredivisie,NAC Breda,Feyenoord,Under
3,NAC vs Feyenoord,Feyenoord 1e Helft: Aantal Goals Over/Under 0.5,Over,1.50,1,2,HL,1,Voetbal,Nederland Eredivisie,NAC vs Feyenoord,NAC,Feyenoord,2025-02-15T17:45:00Z,M,NAC Breda vs Feyenoord,100.0,Goals,1e Helft,NAC,Feyenoord,2,0.5,NaN,NaN,2492348909,1003194956,Totaal Aantal Doelpunten door Feyenoord 1e H...,Total Goals by Feyenoord 1st Half,NaN,NaN,6,Over/Onder,Over/Under,1021049420,3637367916,Minder dan,Under,2.50,NaN,OT_UNDER,2025-02-09T07:44:04Z,6/4,150.0,OPEN,ENABLED,NaN,NaN,NAC Breda vs Feyenoord,FOOTBALL,Eredivisie,NAC Breda,Feyenoord,Over
4,Ipswich Town vs Tottenham Hotspur,Aantal Goals Over/Under 2.5,Under,2.50,6,4,HL,L,Voetbal,Engeland Premier League,Ipswich Town vs Tottenham Hotspur,Ipswich Town,Tottenham Hotspur,2025-02-22T15:00:00Z,M,Ipswich Town vs Tottenham,100.0,Goals,Full Time,Ipswich Town,Tottenham Hotspur,Total team 1 and team 2,2.5,NaN,NaN,2490999429,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021042754,3632068574,Meer dan,Over,1.50,NaN,OT_OVER,2025-02-07T10:45:55Z,1/2,-200.0,OPEN,ENABLED,NaN,NaN,Ipswich Town vs Tottenham,FOOTBALL,Premier League,Ipswich Town,Tottenham,Under
7,Ipswich Town vs Tottenham Hotspur,Aantal Goals Over/Under 2.5,Over,1.47,47,100,HL,1,Voetbal,Engeland Premier League,Ipswich Town vs Tottenham Hotspur,Ipswich Town,Tottenham Hotspur,2025-02-22T15:00:00Z,M,Ipswich Town vs Tottenham,100.0,Goals,Full Time,Ipswich Town,Tottenham Hotspur,Total team 1 and team 2,2.5,NaN,NaN,2490999429,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021042754,3632068575,Minder dan,Under,2.60,NaN,OT_UNDER,2025-02-07T10:45:55Z,8/5,160.0,OPEN,ENABLED,NaN,NaN,Ipswich Town vs Tottenham,FOOTBALL,Premier League,Ipswich Town,Tottenham,Over
8,Ipswich Town vs Tottenham Hotspur,Tottenham Hotspur Aantal Goals Over/Under 1.5,Under,2.10,11,10,HL,L,Voetbal,Engeland Premier League,Ipswich Town vs Tottenham Hotspur,Ipswich Town,Tottenham Hotspur,2025-02-22T15:00:00Z,M,Ipswich Town vs Tottenham,100.0,Goals,Full Time,Ipswich Town,Tottenham Hotspur,2,1.5,NaN,NaN,2490999499,1001159633,Totaal Aantal Doelpunten door Tottenham,Total Goals by Tottenham,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021042754,3632068724,Meer dan,Over,1.65,NaN,OT_OVER,2025-02-07T10:45:55Z,13/20,-155.0,OPEN,ENABLED,NaN,NaN,Ipswich Town vs Tottenham,FOOTBALL,Premier League,Ipswich Town,Tottenham,Under


In [386]:
merged_football_overunder['OverUnderType']

0       Goals
3       Goals
4       Goals
7       Goals
8       Goals
        ...  
6585    Goals
6588    Goals
6589    Goals
6592    Goals
6593    Goals
Name: OverUnderType, Length: 3254, dtype: object

## Match Winnaar & OverUnder and do Arb Analysis

In [387]:
# Perform the stacked union
total_football = pd.concat([merged_football_overunder, merged_df_winnaar], ignore_index=True, sort=True)
total_football

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome Name cleaned,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,fuzzy_score_x,fuzzy_score_y,group_name,home_score,home_team,lifetime,line,match_name,matched_OverUnderType2,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,start_time,status,type
0,NAC vs Feyenoord,Feyenoord 1e Helft: Aantal Goals Over/Under 0.5,2.41,Under,Under,L,HL,1e Helft,Goals,2,100,141,NAC,NAC Breda,Feyenoord,Feyenoord,NaN,Feyenoord,2.492349e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-09T07:44:04Z,Nederland Eredivisie,Total Goals by Feyenoord 1st Half,1.003195e+09,Totaal Aantal Doelpunten door Feyenoord 1e H...,1.021049e+09,NAC Breda vs Feyenoord,NaN,100.0,NaN,Eredivisie,NaN,NAC,NaN,0.5,NAC vs Feyenoord,NaN,NAC Breda vs Feyenoord,NaN,1.46,-220.0,4/9,Over,3.637368e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-15T17:45:00Z,OPEN,OT_OVER
1,NAC vs Feyenoord,Feyenoord 1e Helft: Aantal Goals Over/Under 0.5,1.50,Over,Over,1,HL,1e Helft,Goals,2,2,1,NAC,NAC Breda,Feyenoord,Feyenoord,NaN,Feyenoord,2.492349e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-09T07:44:04Z,Nederland Eredivisie,Total Goals by Feyenoord 1st Half,1.003195e+09,Totaal Aantal Doelpunten door Feyenoord 1e H...,1.021049e+09,NAC Breda vs Feyenoord,NaN,100.0,NaN,Eredivisie,NaN,NAC,NaN,0.5,NAC vs Feyenoord,NaN,NAC Breda vs Feyenoord,NaN,2.50,150.0,6/4,Under,3.637368e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-15T17:45:00Z,OPEN,OT_UNDER
2,Ipswich Town vs Tottenham Hotspur,Aantal Goals Over/Under 2.5,2.50,Under,Under,L,HL,Full Time,Goals,Total team 1 and team 2,4,6,Ipswich Town,Ipswich Town,Tottenham Hotspur,Tottenham,NaN,Tottenham Hotspur,2.490999e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-07T10:45:55Z,Engeland Premier League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021043e+09,Ipswich Town vs Tottenham,NaN,100.0,NaN,Premier League,NaN,Ipswich Town,FULL_TIME,2.5,Ipswich Town vs Tottenham Hotspur,NaN,Ipswich Town vs Tottenham,GOALS,1.50,-200.0,1/2,Over,3.632069e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-22T15:00:00Z,OPEN,OT_OVER
3,Ipswich Town vs Tottenham Hotspur,Aantal Goals Over/Under 2.5,1.47,Over,Over,1,HL,Full Time,Goals,Total team 1 and team 2,100,47,Ipswich Town,Ipswich Town,Tottenham Hotspur,Tottenham,NaN,Tottenham Hotspur,2.490999e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-07T10:45:55Z,Engeland Premier League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021043e+09,Ipswich Town vs Tottenham,NaN,100.0,NaN,Premier League,NaN,Ipswich Town,FULL_TIME,2.5,Ipswich Town vs Tottenham Hotspur,NaN,Ipswich Town vs Tottenham,GOALS,2.60,160.0,8/5,Under,3.632069e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-22T15:00:00Z,OPEN,OT_UNDER
4,Ipswich Town vs Tottenham Hotspur,Tottenham Hotspur Aantal Goals Over/Under 1.5,2.10,Under,Under,L,HL,Full Time,Goals,2,10,11,Ipswich Town,Ipswich Town,Tottenham Hotspur,Tottenham,NaN,Tottenham Hotspur,2.490999e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-07T10:45:55Z,Engeland Premier League,Total Goals by Tottenham,1.001160e+09,Totaal Aantal Doelpunten door Tottenham,1.021043e+09,Ipswich Town vs Tottenham,NaN,100.0,NaN,Premier League,NaN,Ipswich Town,FULL_TIME,1.5,Ipswich Town vs Tottenham Hotspur,NaN,Ipswich Town vs Tottenham,NaN,1.65,-155.0,13/20,Over,3.632069e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-22T15:00:00Z,OPEN,OT_OVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5

In [388]:
# Calculate Arbitrage Percentage
total_football['Arbitrage Percentage'] = (1 / total_football['Odds (Decimal)'] + 1 / total_football['odds']) * 100

# Identify Arbitrage Opportunities
total_football['Is Arbitrage'] = total_football['Arbitrage Percentage'] < 100

# Calculate Optimal Stakes if Arbitrage
total_stake = 1000
total_football['Stake A'] = total_football.apply(
    lambda row: (1 / row['Odds (Decimal)'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)
total_football['Stake B'] = total_football.apply(
    lambda row: (1 / row['odds'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)

total_football[['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]    

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
0,NAC vs Feyenoord,Feyenoord 1e Helft: Aantal Goals Over/Under 0.5,Under,Meer dan,2.41,1.46,109.986927,False,0,0
1,NAC vs Feyenoord,Feyenoord 1e Helft: Aantal Goals Over/Under 0.5,Over,Minder dan,1.50,2.50,106.666667,False,0,0
2,Ipswich Town vs Tottenham Hotspur,Aantal Goals Over/Under 2.5,Under,Meer dan,2.50,1.50,106.666667,False,0,0
3,Ipswich Town vs Tottenham Hotspur,Aantal Goals Over/Under 2.5,Over,Minder dan,1.47,2.60,106.488749,False,0,0
4,Ipswich Town vs Tottenham Hotspur,Tottenham Hotspur Aantal Goals Over/Under 1.5,Under,Meer dan,2.10,1.65,108.225108,False,0,0
...,...,...,...,...,...,...,...,...,...,...
5027,TS Galaxy FC vs Mamelodi Sundowns,2e Helft: Draw No Bet,TS Galaxy FC,2,3.15,1.25,111.746032,False,0,0
5028,FC Uta Arad vs CFR Cluj,2e Helft: Draw No Bet,CFR Cluj,1,1.44,2.55,108.660131,False,0,0
5029,FC Uta Arad vs CFR Cluj,2e Helft: Draw No Bet,FC Uta Arad,2,2.50,1.36,113.529412,False,0,0
5030,TS Galaxy FC vs Mamelodi Sundowns,1e Helft: Draw No Bet,Mamelodi Sundowns,1,1.25,3.20,111.250000,False,0,0


In [389]:
total_football.drop_duplicates().sort_values(by='Arbitrage Percentage', ascending=True)

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome Name cleaned,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,fuzzy_score_x,fuzzy_score_y,group_name,home_score,home_team,lifetime,line,match_name,matched_OverUnderType2,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,start_time,status,type,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
1085,Valencia vs Leganes,Aantal Goals Over/Under 3.5,1.22,Under,Under,L,HL,Full Time,Goals,Total team 1 and team 2,50,11,Valencia,Valencia,Leganes,Leganes,NaN,Leganes,2.489052e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-09T09:55:35Z,Spanje LaLiga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021044e+09,Valencia vs Leganes,NaN,100.0,NaN,La Liga,NaN,Valencia,FULL_TIME,3.5,Valencia vs Leganes,NaN,Valencia vs Leganes,GOALS,5.10,410.0,4/1,Over,3.625082e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-09T15:15:00Z,OPEN,OT_OVER,101.575056,False,0,0
4030,Venezia vs AS Roma,Draw No Bet,1.41,AS Roma,NaN,2,DN,NaN,NaN,NaN,100,41,Venezia,Venezia,AS Roma,Roma,NaN,AS Roma,2.489277e+09,Match,2.0,Wedstrijd,DISABLED,2025-02-09T09:39:10Z,Italië Serie A,Draw No Bet,1.001160e+09,Draw No Bet,1.021152e+09,Venezia vs Roma,100.0,NaN,NaN,Serie A,NaN,Venezia,FULL_TIME,NaN,Venezia vs AS Roma,NaN,Venezia vs Roma,GOALS,3.25,225.0,9/4,1,3.625785e+09,1,Venezia,M,Voetbal,FOOTBALL,Draw No Bet,2025-02-09T11:30:00Z,OPEN,1,101.691217,False,0,0
4997,FCSB vs ACS Sepsi Osk Sfantu Gheorghe,Draw No Bet,1.31,FCSB,NaN,1,DN,NaN,NaN,NaN,100,31,FCSB,FCSB,ACS Sepsi Osk Sfantu Gheorghe,ACS Sepsi OSK Sfantul Gheorghe,NaN,ACS Sepsi Osk Sfantu Gheorghe,2.490798e+09,Match,2.0,Wedstrijd,DISABLED,2025-02-09T05:52:29Z,Roemenië Liga 1,Draw No Bet,1.001160e+09,Draw No Bet,1.021130e+09,FCSB vs ACS Sepsi OSK Sfantul Gheorghe,99.0,NaN,NaN,Liga I,NaN,FCSB,FULL_TIME,NaN,FCSB vs ACS Sepsi Osk Sfantu Gheorghe,NaN,FCSB vs ACS Sepsi OSK Sfantul Gheorghe,GOALS,3.90,290.0,29/10,2,3.631255e+09,2,ACS Sepsi OSK Sfântul Gheorghe,M,Voetbal,FOOTBALL,Draw No Bet,2025-02-09T18:00:00Z,OPEN,2,101.976904,False,0,0
3884,Napoli vs Udinese,Draw No Bet,1.17,Napoli,NaN,1,DN,NaN,NaN,NaN,100,17,Napoli,Napoli,Udinese,Udinese,NaN,Udinese,2.489288e+09,Match,2.0,Wedstrijd,DISABLED,2025-02-09T10:12:44Z,Italië Serie A,Draw No Bet,1.001160e+09,Draw No Bet,1.021152e+09,Napoli vs Udinese,100.0,NaN,NaN,Serie A,NaN,Napoli,FULL_TIME,NaN,Napoli vs Udinese,NaN,Napoli vs Udinese,GOALS,6.00,500.0,5/1,2,3.625750e+09,2,Udinese,M,Voetbal,FOOTBALL,Draw No Bet,2025-02-09T19:45:00Z,OPEN,2,102.136752,False,0,0
1310,Cagliari vs Parma Calcio,Aantal Goals Over/Under 4.5,1.18,Under,Under,L,HL,Full Time,Goals,Total team 1 and team 2,50,9,Cagliari,Cagliari,Parma Calcio,Parma,NaN,Parma Calcio,2.489271e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-09T09:11:36Z,Italië Serie A,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021152e+09,Cagliari vs Parma,NaN,100.0,NaN,Serie A,NaN,Cagliari,FULL_TIME,4.5,Cagliari vs Parma Calcio,NaN,Cagliari vs Parma,GOALS,5.75,475.0,19/4,Over,3.625735e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-09T14:00:00Z,OPEN,OT_OVER,102.137067,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941,Guadalajara Chivas vs Club Tijuana De Caliente,1e Helft: Draw No Bet,2.80,Club Tijuana De Caliente,NaN,2,DN,NaN,NaN,NaN,5,9,Guadalajara Chivas,NaN,Club Tijuana De Caliente,NaN,NaN,Club Tijuana De Caliente,NaN,NaN,NaN,NaN,NaN,NaN,Mexico Liga MX,NaN,NaN,N

In [390]:
total_football[total_football['Event Name'].str.contains('Rakow Czestochowa vs GKS Katowice')]

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome Name cleaned,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,fuzzy_score_x,fuzzy_score_y,group_name,home_score,home_team,lifetime,line,match_name,matched_OverUnderType2,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,start_time,status,type,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B


In [391]:
toto_raw_football[(toto_raw_football['Event Name'].str.contains('Rakow Czestochowa vs GKS Katowice')) & (toto_raw_football['Market Name'].str.contains('Aantal Goals - Over/Under 3.5'))]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex
event_id,,,,,,,,,,,,,,,


In [392]:
toto_filtered_football[(toto_filtered_football['Event Name'].str.contains('Rakow Czestochowa vs GKS Katowice')) & (toto_filtered_football['Market Name'].str.contains('Aantal Goals - Over/Under 3.5'))]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex
event_id,,,,,,,,,,,,,,,


In [393]:
total_football[total_football['Is Arbitrage'] == True][['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]  

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B


In [394]:
total_football[(total_football['Is Arbitrage'] == True)][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'Outcome Name cleaned', 'line', 'outcome_english_label', 'type', 'Outcome SubType', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderType2', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex', 'start_time']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,Outcome Name cleaned,line,outcome_english_label,type,Outcome SubType,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderType2,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex,start_time


In [395]:
total_football[total_football['OverUnderType'] == 'Speler aantal schoten op doel'][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'Outcome Name cleaned', 'line', 'outcome_english_label', 'type', 'Outcome SubType', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex', 'start_time']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,Outcome Name cleaned,line,outcome_english_label,type,Outcome SubType,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex,start_time


In [396]:
# # Function to find the best fuzzy match for OverUnderType2
# def find_best_fuzzy_match(overunder_type2, kambi_df):
#     # Perform fuzzy matching with all Kambi OverUnderType2 values
#     match_scores = kambi_df['OverUnderType2'].apply(lambda x: fuzz.token_set_ratio(overunder_type2, x))
    
#     # Get the best match score and the corresponding OverUnderType2 value
#     best_match_idx = match_scores.idxmax()
#     best_score = match_scores.max()
    
#     if best_score >= 65:  # Threshold for fuzzy matching
#         return kambi_df.loc[best_match_idx, 'OverUnderType2'], best_score
#     return None, None  # No match found

# # Filter records where OverUnderType contains 'Team' or 'Speler'
# filtered_toto = toto_filtered_football_overunder[
#     toto_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]
# filtered_kambi = kambi_filtered_football_overunder[
#     kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]

# # Apply fuzzy matching function
# filtered_toto[['matched_OverUnderType2', 'fuzzy_score']] = filtered_toto['OverUnderType2'].apply(
#     lambda x: pd.Series(find_best_fuzzy_match(x, filtered_kambi))
# )

# # Merge back to the original dataset (if needed)
# toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
#     filtered_toto[['OverUnderType2', 'matched_OverUnderType2', 'fuzzy_score']],
#     on='OverUnderType2',
#     how='left'
# )

In [397]:
# Fill matched_OverUnderType2 with OverUnderType if it is null
toto_filtered_football_overunder['final_OverUnderType'] = toto_filtered_football_overunder.apply(
    lambda row: row['OverUnderType'] if pd.isna(row['matched_OverUnderType2']) else row['matched_OverUnderType2'], axis=1
)

In [398]:
toto_filtered_football_overunder[toto_filtered_football_overunder['matched_OverUnderType2'].isna()]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score_x,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2,line,matched_OverUnderType2,fuzzy_score_y,final_OverUnderType
0,Oxford United vs Portsmouth,Portsmouth 1e Helft: Aantal Goals Over/Under...,Under,1.70,7,10,HL,L,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z,M,None,NaN,Goals,1e Helft,Oxford United,Portsmouth,2,0.5,NaN,NaN,Goals
1,Oxford United vs Portsmouth,Portsmouth 1e Helft: Aantal Goals Over/Under...,Over,2.03,103,100,HL,1,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z,M,None,NaN,Goals,1e Helft,Oxford United,Portsmouth,2,0.5,NaN,NaN,Goals
2,Oxford United vs Portsmouth,Portsmouth Aantal Goals Over/Under 1.5,Over,2.30,13,10,HL,1,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z,M,None,NaN,Goals,Full Time,Oxford United,Portsmouth,2,1.5,NaN,NaN,Goals
3,Oxford United vs Portsmouth,Portsmouth Aantal Goals Over/Under 1.5,Under,1.54,27,50,HL,L,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z,M,None,NaN,Goals,Full Time,Oxford United,Portsmouth,2,1.5,NaN,NaN,Goals
4,Oxford United vs Portsmouth,Resultaat en Aantal Goals Over/Under 2.5,Gelijkspel en Under,4.25,13,4,--,NaN,Voetbal,Engeland Championship,Oxford United vs Portsmouth,Oxford United,Portsmouth,2025-02-15T12:30:00Z,M,None,NaN,Dubbele Kans,Full Time,Oxford United,Portsmouth,other,2.5,NaN,NaN,Dubbele Kans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135101,IFK Varnamo vs IK Sirius,1e Helft: Aantal Goals Over/Under 2.5,Over,6.50,11,2,HL,1,Voetbal,Zweden Allsvenskan,IFK Värnamo vs IK Sirius,IFK Värnamo,IK Sirius,2025-03-31T17:00:00Z,M,None,NaN,Goals,1e Helft,IFK Varnamo,IK Sirius,Total team 1 and team 2,2.5,NaN,NaN,Goals
135102,IFK Varnamo vs IK Sirius,1e Helft: Aantal Goals Over/Under 0.5,Over,1.40,2,5,HL,1,Voetbal,Zweden Allsvenskan,IFK Värnamo vs IK Sirius,IFK Värnamo,IK Sirius,2025-03-31T17:00:00Z,M,None,NaN,Goals,1e Helft,IFK Varnamo,IK Sirius,Total team 1 and team 2,0.5,NaN,NaN,Goals
135103,IFK Varnamo vs IK Sirius,1e Helft: Aantal Goals Over/Under 0.5,Under,2.70,17,10,HL,L,Voetbal,Zweden Allsvenskan,IFK Värnamo vs IK Sirius,IFK Värnamo,IK Sirius,2025-03-31T17:00:00Z,M,None,NaN,Goals,1e Helft,IFK Varnamo,IK Sirius,Total team 1 and team 2,0.5,NaN,NaN,Goals
135104,IFK Varnamo vs IK Sirius,IK Sirius 2e Helft: Aantal Goals Over/Under 0.5,Over,1.87,87,100,HL,1,Voetbal,Zweden Allsvenskan,IFK Värnamo vs IK Sirius,IFK Värnamo,IK Sirius,2025-03-31T17:00:00Z,M,None,NaN,Goals,2e Helft,IFK Varnamo,IK Sirius,2,0.5,NaN,NaN,Goals
